In [1]:
!pip3 install --upgrade --user google-cloud-aiplatform --quiet

# Restart the kernel before proceeding

In [3]:
import time
from vertexai.generative_models import FunctionDeclaration, GenerativeModel, Part, Tool
import os
import requests
import time


In [18]:
# Authenticate your notebook environment (Colab only)
# If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using Vertex AI Workbench.

from google.colab import auth

auth.authenticate_user()

In [5]:
#Get the weather API Key
weather_api_key = "<Your API KEY>"  #get a key from https://weatherbit.io


In [19]:
PROJECT_ID = "<Your PROJECT>"
LOCATION = "us-central1"

# Initialize Vertex AI
import vertexai
import time

vertexai.init(project=PROJECT_ID, location=LOCATION)


In [24]:
# This function gets the current weather for a given location using the WeatherBit API
def get_current_weather(location: str, lat: float, lon: float):
    # Make a GET request to the WeatherBit API with the provided latitude and longitude
    response = requests.get(f"https://api.weatherbit.io/v2.0/current?lat={lat}&lon={lon}&key={api_key}&include=minutely")

    # Convert the response to a JSON object
    response = response.json()

    # Construct a string with the current weather description and temperature for the given location
    api_response = "The current weather in " + location + " is " + response["data"][0]["weather"]["description"] + " with a temperature of " + str(response["data"][0]["temp"]) + "°C."

    # Return the constructed string
    return api_response

# This function gets the current time and date for a given time zone using the World Time API
def get_current_time(time_zone: str):
    # Make a GET request to the World Time API with the provided time zone
    response = requests.get(f"http://worldtimeapi.org/api/timezone/{time_zone}")

    # Convert the response to a JSON object
    response = response.json()
    print(response)

    # Construct a string with the current time and date for the given time zone
    api_response = "The current time in " + time_zone + " is " + response["datetime"] + ". Today's date is " + response["datetime"][0:10] + "."

    # Return the constructed string
    return api_response


In [14]:
# Function Declaration

get_current_weather_func = FunctionDeclaration(
    name="get_current_weather",
    description="Get the current weather in a given location. Use your knowledge of latitude and longitude to get the weather for a location.",
    parameters={
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The location to get the weather for. For example, 'New York, NY' or 'London, UK'",
            },
            "latitude": {
                "type": "number",
                "description": "The latitude of the location to get the weather for",
            },
            "longitude": {
                "type": "number",
                "description": "The longitude of the location to get the weather for",
            },

        },
        "required": ["location", "latitude", "longitude"],
    },

    )

# Function Declaration

get_current_time_func = FunctionDeclaration(
    name="get_current_time",
    description="Get the current time and date in a given time zone. Use this tool before googling questions that are time sensitive.",
    parameters={
        "type": "object",
        "properties": {
            "time_zone": {
                "type": "string",
                "description": "The time zone to get the time for. For example, 'America/New_York' or 'Europe/London'",
            },
        },
        "required": ["time_zone"],
    },

    )

sql_query_tool = Tool(
    function_declarations=[
        # google_search_func,
        get_current_weather_func,
        get_current_time_func,
    ],
)

In [26]:
# Initialize the Vertex AI GenerativeModel with the specified model name, generation configuration, and tools
model = GenerativeModel(
    "gemini-1.5-pro-preview-0409",
    generation_config={"temperature": 0},
    tools=[sql_query_tool],
)


In [ ]:

# This is the main function that handles user input and generates a response
def main(message):

    full_response = ""
    # Get the current date and time
    current_date = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    # Start a new chat with the model
    chat = model.start_chat(response_validation=False)

    # Construct the prompt for the model, including the user's question and instructions for using the available tools
    prompt = "User Question: " + message + "\n"

    prompt += f"""
        You are Gemini, a large language model trained by Google. Current date: {current_date}

        You have the tool to be used in the following circumstances:
        - User is asking about current events or something that requires real-time information (weather, sports scores, etc.)
        - User is asking about some term you are totally unfamiliar with (it might be new)
        - User explicitly asks you to browse the web for information. Note: The more specific the query, the more accurate the results.
        For example, if user asks 'When does Google Next Happen?' the query should be 'Google Next 2024 date'.
        ALWAYS refine the user question to get the most accurate results, NEVER use the user question as is.
        If you are unsatisfied with the original results retry with a better query.
        Think Step by Step: 1. Understand the user question 2. Refine the query 3. Get the search results 4. Summarize the search results in a concise manner.
        """

    # Send the prompt to the model and get the initial response
    response = chat.send_message(prompt)
    response = response.candidates[0].content.parts[0]

    print(response)

    api_requests_and_responses = []
    backend_details = ""

    function_calling_in_process = True
    while function_calling_in_process:
        try:
            params = {}
            # Extract the function call parameters from the response
            for key, value in response.function_call.args.items():
                params[key] = value

            print(response.function_call.name)
            print(params)

            # If the response is a call to the get_current_weather function, execute it with the provided parameters
            if response.function_call.name == "get_current_weather":
                location = params["location"]
                lat = params["latitude"]
                lon = params["longitude"]
                api_response = get_current_weather(location, lat, lon)
                api_requests_and_responses.append(
                    [response.function_call.name, params, api_response]
                )

            # If the response is a call to the get_current_time function, execute it with the provided parameters
            if response.function_call.name == "get_current_time":
                time_zone = params["time_zone"]
                api_response = get_current_time(time_zone)
                api_requests_and_responses.append(
                    [response.function_call.name, params, api_response]
                )

            print(api_response)

            # Send the API response back to the model and get the next response
            response = chat.send_message(
                Part.from_function_response(
                    name=response.function_call.name,
                    response={
                        "content": api_response,
                    },
                ),
            )
            response = response.candidates[0].content.parts[0]

            # Construct a string with details of the function call, parameters, and API response
            backend_details += "- Function call:\n"
            backend_details += (
                "   - Function name: ```"
                + str(api_requests_and_responses[-1][0])
                + "```"
            )
            backend_details += "\n\n"
            backend_details += (
                "   - Function parameters: ```"
                + str(api_requests_and_responses[-1][1])
                + "```"
            )
            backend_details += "\n\n"
            backend_details += (
                "   - API response: ```"
                + str(api_requests_and_responses[-1][2])
                + "```"
            )
            backend_details += "\n\n"

        except AttributeError:
            # If there is no more function call in the response, exit the loop
            function_calling_in_process = False

    # Wait for 3 seconds (not sure why)
    time.sleep(3)
    text_content = backend_details

    # Get the final response text from the model
    full_response = response.text
    print(full_response)



In [30]:
# Call the main function with a user message
main("What's the weather like in New York City today?")

function_call {
  name: "get_current_weather"
  args {
    fields {
      key: "latitude"
      value {
        number_value: 40.7128
      }
    }
    fields {
      key: "location"
      value {
        string_value: "New York City"
      }
    }
    fields {
      key: "longitude"
      value {
        number_value: -74.006
      }
    }
  }
}

get_current_weather
{'longitude': -74.006, 'latitude': 40.7128, 'location': 'New York City'}
The current weather in New York City is Clear sky with a temperature of 11.4°C.
The current weather in New York City is Clear sky with a temperature of 11.4°C. 



In [29]:
main("What's the time New York City now?")

function_call {
  name: "get_current_time"
  args {
    fields {
      key: "time_zone"
      value {
        string_value: "America/New_York"
      }
    }
  }
}

get_current_time
{'time_zone': 'America/New_York'}
{'abbreviation': 'EDT', 'client_ip': '34.16.166.157', 'datetime': '2024-04-22T22:11:37.340752-04:00', 'day_of_week': 1, 'day_of_year': 113, 'dst': True, 'dst_from': '2024-03-10T07:00:00+00:00', 'dst_offset': 3600, 'dst_until': '2024-11-03T06:00:00+00:00', 'raw_offset': -18000, 'timezone': 'America/New_York', 'unixtime': 1713838297, 'utc_datetime': '2024-04-23T02:11:37.340752+00:00', 'utc_offset': '-04:00', 'week_number': 17}
The current time in America/New_York is 2024-04-22T22:11:37.340752-04:00. Today's date is 2024-04-22.
The current time in New York City is 10:11 PM on April 22, 2024. 

